# Using the Dataloader

In [1]:
import numpy as np
import pandas as pd
import copy
import torch
import sys
import os
from pgmpy.models import BayesianModel
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.estimators import BayesianEstimator
from pgmpy.inference import VariableElimination
from torch.utils.data import Dataset, DataLoader
sys.path.append("./..")
import classes.powerdf as pdf
import matplotlib.pyplot as plt
from classes.data_loader import smarthome_dataset

/Users/meezy/.pyenv/versions/3.11.5/envs/jupyterenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cur_path = os.path.abspath(os.getcwd())
dataset_path = os.path.join(cur_path,"cleaned_HomeC.csv")
print(dataset_path)
df = pd.read_csv(dataset_path, low_memory = False)
dataset = smarthome_dataset("cleaned_HomeC.csv" , dayahead = False)
print("dataset length", len(dataset))

/Users/meezy/Documents/Carnegie/10-708_F23/GRIDPredict/notebooks/cleaned_HomeC.csv
dataset length 503910


In [3]:
# create training and testing dataset
# fill missing values in
df.fillna(value = 0)
train_df = df.iloc[:3000]
test_df = df.iloc[4000:4400]
df = df.iloc[:]

traindata_loader = DataLoader(dataset, batch_size=10, shuffle=True,
                              num_workers=0)
testdata_loader = DataLoader(dataset, batch_size=10, shuffle=True,
                             num_workers=0)

"""
open = True
while (open):
    try:
        inputs, label = next(iter(traindata_loader))  
        print(inputs.shape)
        print(label.shape)
        #print("input {} : label {}".format(inputs,label))
        open = True
    except:
        open = False
"""

'\nopen = True\nwhile (open):\n    try:\n        inputs, label = next(iter(traindata_loader))  \n        print(inputs.shape)\n        print(label.shape)\n        #print("input {} : label {}".format(inputs,label))\n        open = True\n    except:\n        open = False\n'

The data loader returns input features corresponding to the weather features and the label corresponds to the label.

If dayahead is True (predicting next hour's solar generation based on previous 24 hourly weather features)...

\begin{align}
\text{Input Size}: & (\text{batch size } \times \text{ weather features } \times \text{ past 24 hour data points})\\
\text{Labels}: & (\text{batch size})
\end{align}

If dayahead is False (predicting hours solar generation based on current weather features)

\begin{align}
\text{Input Size}: & (\text{batch size } \times \text{ weather features })\\
\text{Labels}: & (\text{batch size})
\end{align}

In [4]:
# Using MLP
from networks.mlp_baseline import *
mlp_model = MLP_baseline(1,9)
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.001)
loss_fn = kl_sum_loss

print("Beginning Training...")
print("data loader size", len(traindata_loader))
num_epochs = 1
for n in range(num_epochs):
    total_loss = 0
    for batch_idx, (x, y) in enumerate(traindata_loader):
        print("batch {}".format(batch_idx))
        x, y = next(iter(traindata_loader))
        print(x.shape)
        print(x)
        y_pred = mlp_model(x)
        loss = loss_fn(y_pred, y)
        total_loss = loss + total_loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (batch_idx > 500):
            break
    print("epoch loss:", loss.item())

torch.save(mlp_model, "mlp.pth")

Beginning Training...
data loader size 50391
batch 0
torch.Size([10, 9])
tensor([[1.2120e+01, 2.7000e-01, 3.0100e+02, 0.0000e+00, 2.1360e+01, 1.0022e+03,
         2.2320e+01, 9.8200e+00, 6.5000e-01],
        [2.6400e+00, 5.0000e-02, 1.9100e+02, 0.0000e+00, 6.6630e+01, 1.0190e+03,
         8.3850e+01, 9.8000e+00, 6.1000e-01],
        [4.2800e+00, 9.0000e-02, 3.3000e+01, 0.0000e+00, 5.5210e+01, 1.0228e+03,
         6.1900e+01, 1.0000e+01, 7.9000e-01],
        [2.4000e+00, 5.0000e-02, 1.7900e+02, 0.0000e+00, 2.1890e+01, 1.0200e+03,
         2.6540e+01, 9.6200e+00, 8.2000e-01],
        [5.9000e+00, 2.3000e-01, 2.2800e+02, 0.0000e+00, 3.4530e+01, 1.0121e+03,
         3.9350e+01, 1.0000e+01, 7.2000e-01],
        [2.2800e+00, 0.0000e+00, 1.8500e+02, 0.0000e+00, 3.7500e+01, 1.0253e+03,
         3.9470e+01, 8.5600e+00, 9.3000e-01],
        [5.7100e+00, 0.0000e+00, 2.8500e+02, 0.0000e+00, 6.7970e+01, 1.0165e+03,
         7.4600e+01, 1.0000e+01, 8.0000e-01],
        [1.1510e+01, 1.0000e+00, 2.800

In [5]:
# evaluation
error_terms = np.array([])
for batch_idx, (x, y) in enumerate(testdata_loader):
    print("batch {}".format(batch_idx))
    x, y = next(iter(testdata_loader))
    print(y)
    print(y_pred)
    y_pred = mlp_model(x)
    se_loss = (y - y_pred) * (y - y_pred)
    print("{} squared error loss {}".format(batch_idx, se_loss))
    for term in se_loss:
        np.append(term, se_loss.detach().numpy())
print("mean squared error:", np.mean(se_loss))

batch 0
tensor([ 3.9630,  0.1864, 15.7970,  0.2537,  8.9964,  0.2006, 12.5966,  0.1843,
        11.0642,  0.1881])
tensor([[3.1411],
        [3.0519],
        [2.9315],
        [3.5910],
        [2.7860],
        [3.2091],
        [3.7713],
        [3.4117],
        [3.8549],
        [2.8131]], grad_fn=<AddmmBackward0>)
0 squared error loss tensor([[2.1584e-01, 1.7988e+01, 1.2926e+02, 1.7421e+01, 2.0874e+01, 1.7867e+01,
         6.6734e+01, 1.8005e+01, 4.4045e+01, 1.7973e+01],
        [3.3090e-02, 1.2922e+01, 1.4438e+02, 1.2442e+01, 2.7200e+01, 1.2819e+01,
         7.7714e+01, 1.2936e+01, 5.3044e+01, 1.2909e+01],
        [2.3419e-01, 1.8152e+01, 1.2882e+02, 1.7583e+01, 2.0698e+01, 1.8031e+01,
         6.6418e+01, 1.8170e+01, 4.3789e+01, 1.8137e+01],
        [2.6441e-01, 1.0643e+01, 1.5248e+02, 1.0209e+01, 3.0776e+01, 1.0550e+01,
         8.3684e+01, 1.0656e+01, 5.7995e+01, 1.0632e+01],
        [1.2995e-03, 1.4536e+01, 1.3919e+02, 1.4028e+01, 2.4974e+01, 1.4428e+01,
         7.3919e+01,

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.